In [1]:
# Select TensorFlow 2.0 environment (works only on Colab)
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [0]:
# Install wandb (ignore if already done)
!pip install wandb

In [0]:
# Authorize wandb
!wandb login

In [0]:
# Imports
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from wandb.keras import WandbCallback
import tensorflow as tf
import numpy as np
import wandb
import time

In [0]:
# Fix the random generator seeds for better reproducibility
tf.random.set_seed(67)
np.random.seed(67)

In [0]:
# Load the dataset
fashion_mnist = tf.keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# Scale the pixel values of the images to 
train_images = train_images / 255.0
test_images = test_images / 255.0

# Reshape the pixel values so that they are compatible with
# the conv layers
train_images = train_images.reshape(-1, 28, 28, 1)
test_images = test_images.reshape(-1, 28, 28, 1)

In [0]:
# Specify the labels of FashionMNIST dataset, it would
# be needed later 😉
labels = ["T-shirt/top","Trouser","Pullover","Dress","Coat",
        "Sandal","Shirt","Sneaker","Bag","Ankle boot"]

In [0]:
METHOD = 'bayes' # change to 'random' or 'bayes' when necessary and rerun

In [0]:
def train():
    # Prepare data tuples
    (X_train, y_train) = train_images, train_labels
    (X_test, y_test) = test_images, test_labels
    
    # Default values for hyper-parameters we're going to sweep over
    configs = {
        'layers': 128,
        'batch_size': 64,
        'epochs': 5,
        'method': METHOD
    }
    
    # Initilize a new wandb run
    wandb.init(project='hyperparameter-sweeps-comparison', config=configs)
    
    # Config is a variable that holds and saves hyperparameters and inputs
    config = wandb.config

    # Add the config items to wandb
    if wandb.run:
        wandb.config.update({k: v for k, v in configs.items() if k not in dict(wandb.config.user_items())})
                           
    # Define the model
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        MaxPooling2D((2,2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2,2)),
        Conv2D(64, (3, 3), activation='relu'),
        GlobalAveragePooling2D(),
        Dense(config.layers, activation=tf.nn.relu),
        Dense(10, activation='softmax')
    ])
    
    # Compile the model
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    # Train the model
    model.fit(X_train, y_train, 
             epochs=config.epochs,
             batch_size=config.batch_size,
             validation_data=(X_test, y_test),
             callbacks=[WandbCallback(data_type="image", 
                validation_data=(X_test, y_test), labels=labels)])

In [0]:
# A function to specify the tuning configuration, it would also
# return us a sweep id (required for running the sweep)
def get_sweep_id(method):
    sweep_config = {
        'method': method,
        'metric': {
          'name': 'accuracy',
          'goal': 'maximize'   
        },
        'parameters': {
            'layers': {
                'values': [32, 64, 96, 128, 256]
            },
            'batch_size': {
                'values': [32, 64, 96, 128]
            },
            'epochs': {
                'values': [5, 10, 15]
            }
        }
    }
    sweep_id = wandb.sweep(sweep_config, project='hyperparameter-sweeps-comparison')
    
    return sweep_id

In [0]:
# Create a sweep for *grid* search
sweep_id = get_sweep_id('grid')

Create sweep with ID: t1kn8296
Sweep URL: https://app.wandb.ai/sayakpaul/hyperparameter-sweeps-comparison/sweeps/t1kn8296


In [0]:
# Run the sweep
wandb.agent(sweep_id, function=train)

wandb: Agent Starting Run: 86mr9bse with config:
	batch_size: 32
	epochs: 5
	layers: 32
wandb: Agent Started Run: 86mr9bse


Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 13s 214us/sample - loss: 0.7193 - accuracy: 0.7288 - val_loss: 0.5529 - val_accuracy: 0.7916
Epoch 2/5
60000/60000 [==============================] - 6s 98us/sample - loss: 0.4613 - accuracy: 0.8317 - val_loss: 0.4271 - val_accuracy: 0.8454
Epoch 3/5
60000/60000 [==============================] - 6s 99us/sample - loss: 0.3829 - accuracy: 0.8606 - val_loss: 0.4076 - val_accuracy: 0.8576
Epoch 4/5
60000/60000 [==============================] - 6s 99us/sample - loss: 0.3397 - accuracy: 0.8760 - val_loss: 0.3742 - val_accuracy: 0.8675
Epoch 5/5
60000/60000 [==============================] - 6s 97us/sample - loss: 0.3119 - accuracy: 0.8869 - val_loss: 0.3322 - val_accuracy: 0.8805
wandb: Agent Finished Run: 86mr9bse 

wandb: Agent Starting Run: oq6yo128 with config:
	batch_size: 32
	epochs: 5
	layers: 64
wandb: Agent Started Run: oq6yo128


Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 8s 130us/sample - loss: 0.6908 - accuracy: 0.7405 - val_loss: 0.4959 - val_accuracy: 0.8211
Epoch 2/5
60000/60000 [==============================] - 6s 98us/sample - loss: 0.4439 - accuracy: 0.8392 - val_loss: 0.4143 - val_accuracy: 0.8495
Epoch 3/5
60000/60000 [==============================] - 6s 98us/sample - loss: 0.3723 - accuracy: 0.8633 - val_loss: 0.3924 - val_accuracy: 0.8603
Epoch 4/5
60000/60000 [==============================] - 6s 99us/sample - loss: 0.3292 - accuracy: 0.8793 - val_loss: 0.3682 - val_accuracy: 0.8670
Epoch 5/5
60000/60000 [==============================] - 6s 99us/sample - loss: 0.2991 - accuracy: 0.8904 - val_loss: 0.3157 - val_accuracy: 0.8897
wandb: Agent Finished Run: oq6yo128 

wandb: Agent Starting Run: ymx4yx9t with config:
	batch_size: 32
	epochs: 5
	layers: 96
wandb: Agent Started Run: ymx4yx9t


Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 8s 137us/sample - loss: 0.6899 - accuracy: 0.7373 - val_loss: 0.5280 - val_accuracy: 0.8010
Epoch 2/5
60000/60000 [==============================] - 6s 98us/sample - loss: 0.4442 - accuracy: 0.8370 - val_loss: 0.4155 - val_accuracy: 0.8495
Epoch 3/5
60000/60000 [==============================] - 6s 98us/sample - loss: 0.3618 - accuracy: 0.8668 - val_loss: 0.4064 - val_accuracy: 0.8549
Epoch 4/5
60000/60000 [==============================] - 6s 97us/sample - loss: 0.3174 - accuracy: 0.8834 - val_loss: 0.3795 - val_accuracy: 0.8608
Epoch 5/5
60000/60000 [==============================] - 6s 98us/sample - loss: 0.2896 - accuracy: 0.8946 - val_loss: 0.3179 - val_accuracy: 0.8862
wandb: Agent Finished Run: ymx4yx9t 

wandb: Agent Starting Run: yikb3oey with config:
	batch_size: 32
	epochs: 5
	layers: 128
wandb: Agent Started Run: yikb3oey


Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 8s 129us/sample - loss: 0.6789 - accuracy: 0.7397 - val_loss: 0.4918 - val_accuracy: 0.8155
Epoch 2/5
60000/60000 [==============================] - 6s 99us/sample - loss: 0.4343 - accuracy: 0.8414 - val_loss: 0.4133 - val_accuracy: 0.8505
Epoch 3/5
60000/60000 [==============================] - 6s 100us/sample - loss: 0.3595 - accuracy: 0.8668 - val_loss: 0.3973 - val_accuracy: 0.8552
Epoch 4/5
60000/60000 [==============================] - 6s 100us/sample - loss: 0.3139 - accuracy: 0.8841 - val_loss: 0.3665 - val_accuracy: 0.8639
Epoch 5/5
60000/60000 [==============================] - 6s 99us/sample - loss: 0.2847 - accuracy: 0.8949 - val_loss: 0.3071 - val_accuracy: 0.8894
wandb: Agent Finished Run: yikb3oey 

wandb: Agent Starting Run: 958bprt0 with config:
	batch_size: 32
	epochs: 5
	layers: 256
wandb: Agent Started Run: 958bprt0


...

Train on 60000 samples, validate on 10000 samples
Epoch 1/15
60000/60000 [==============================] - 5s 76us/sample - loss: 0.9015 - accuracy: 0.6608 - val_loss: 0.6466 - val_accuracy: 0.7498
Epoch 2/15
60000/60000 [==============================] - 2s 41us/sample - loss: 0.5783 - accuracy: 0.7796 - val_loss: 0.5533 - val_accuracy: 0.7977
Epoch 3/15
60000/60000 [==============================] - 2s 42us/sample - loss: 0.4970 - accuracy: 0.8179 - val_loss: 0.4931 - val_accuracy: 0.8238
Epoch 4/15
60000/60000 [==============================] - 2s 41us/sample - loss: 0.4405 - accuracy: 0.8408 - val_loss: 0.4411 - val_accuracy: 0.8454
Epoch 5/15
60000/60000 [==============================] - 2s 40us/sample - loss: 0.4064 - accuracy: 0.8528 - val_loss: 0.4127 - val_accuracy: 0.8522
Epoch 6/15
60000/60000 [==============================] - 2s 39us/sample - loss: 0.3765 - accuracy: 0.8646 - val_loss: 0.4031 - val_accuracy: 0.8535
Epoch 7/15
60000/60000 [==============================] 

Train on 60000 samples, validate on 10000 samples
Epoch 1/15
60000/60000 [==============================] - 4s 72us/sample - loss: 0.8702 - accuracy: 0.6774 - val_loss: 0.6410 - val_accuracy: 0.7488
Epoch 2/15
60000/60000 [==============================] - 2s 37us/sample - loss: 0.5669 - accuracy: 0.7856 - val_loss: 0.5494 - val_accuracy: 0.7986
Epoch 3/15
60000/60000 [==============================] - 2s 40us/sample - loss: 0.4804 - accuracy: 0.8248 - val_loss: 0.4880 - val_accuracy: 0.8305
Epoch 4/15
60000/60000 [==============================] - 2s 39us/sample - loss: 0.4291 - accuracy: 0.8463 - val_loss: 0.4210 - val_accuracy: 0.8523
Epoch 5/15
60000/60000 [==============================] - 2s 40us/sample - loss: 0.3919 - accuracy: 0.8594 - val_loss: 0.3951 - val_accuracy: 0.8617
Epoch 6/15
60000/60000 [==============================] - 2s 39us/sample - loss: 0.3615 - accuracy: 0.8694 - val_loss: 0.3733 - val_accuracy: 0.8657
Epoch 7/15
60000/60000 [==============================] 

Train on 60000 samples, validate on 10000 samples
Epoch 1/15
60000/60000 [==============================] - 4s 73us/sample - loss: 0.8469 - accuracy: 0.6823 - val_loss: 0.6256 - val_accuracy: 0.7551
Epoch 2/15
60000/60000 [==============================] - 2s 39us/sample - loss: 0.5638 - accuracy: 0.7847 - val_loss: 0.5295 - val_accuracy: 0.8044
Epoch 3/15
60000/60000 [==============================] - 2s 39us/sample - loss: 0.4810 - accuracy: 0.8231 - val_loss: 0.4978 - val_accuracy: 0.8210
Epoch 4/15
60000/60000 [==============================] - 2s 39us/sample - loss: 0.4278 - accuracy: 0.8453 - val_loss: 0.4239 - val_accuracy: 0.8482
Epoch 5/15
60000/60000 [==============================] - 2s 38us/sample - loss: 0.3908 - accuracy: 0.8585 - val_loss: 0.4005 - val_accuracy: 0.8567
Epoch 6/15
60000/60000 [==============================] - 2s 38us/sample - loss: 0.3601 - accuracy: 0.8687 - val_loss: 0.4180 - val_accuracy: 0.8481
Epoch 7/15
60000/60000 [==============================] 

Train on 60000 samples, validate on 10000 samples
Epoch 1/15
60000/60000 [==============================] - 4s 72us/sample - loss: 0.8427 - accuracy: 0.6780 - val_loss: 0.6123 - val_accuracy: 0.7654
Epoch 2/15
60000/60000 [==============================] - 2s 38us/sample - loss: 0.5498 - accuracy: 0.7921 - val_loss: 0.5226 - val_accuracy: 0.8077
Epoch 3/15
60000/60000 [==============================] - 2s 39us/sample - loss: 0.4689 - accuracy: 0.8280 - val_loss: 0.4671 - val_accuracy: 0.8352
Epoch 4/15
60000/60000 [==============================] - 2s 39us/sample - loss: 0.4134 - accuracy: 0.8500 - val_loss: 0.4099 - val_accuracy: 0.8530
Epoch 5/15
60000/60000 [==============================] - 2s 39us/sample - loss: 0.3753 - accuracy: 0.8633 - val_loss: 0.3739 - val_accuracy: 0.8660
Epoch 6/15
60000/60000 [==============================] - 2s 39us/sample - loss: 0.3471 - accuracy: 0.8733 - val_loss: 0.3843 - val_accuracy: 0.8631
Epoch 7/15
60000/60000 [==============================] 

Train on 60000 samples, validate on 10000 samples
Epoch 1/15
60000/60000 [==============================] - 4s 73us/sample - loss: 0.8158 - accuracy: 0.6903 - val_loss: 0.6024 - val_accuracy: 0.7670
Epoch 2/15
60000/60000 [==============================] - 2s 39us/sample - loss: 0.5392 - accuracy: 0.7988 - val_loss: 0.5220 - val_accuracy: 0.8122
Epoch 3/15
60000/60000 [==============================] - 2s 39us/sample - loss: 0.4581 - accuracy: 0.8339 - val_loss: 0.4580 - val_accuracy: 0.8375
Epoch 4/15
60000/60000 [==============================] - 2s 39us/sample - loss: 0.4074 - accuracy: 0.8520 - val_loss: 0.4053 - val_accuracy: 0.8547
Epoch 5/15
60000/60000 [==============================] - 2s 38us/sample - loss: 0.3735 - accuracy: 0.8641 - val_loss: 0.3852 - val_accuracy: 0.8619
Epoch 6/15
60000/60000 [==============================] - 2s 38us/sample - loss: 0.3433 - accuracy: 0.8743 - val_loss: 0.3944 - val_accuracy: 0.8566
Epoch 7/15
60000/60000 [==============================] 

In [0]:
# Create a sweep for *random* search (run METHOD cell first and then train())
sweep_id = get_sweep_id('random')

Create sweep with ID: vb7k0mim
Sweep URL: https://app.wandb.ai/sayakpaul/hyperparameter-sweeps-comparison/sweeps/vb7k0mim


In [0]:
# Run the sweep
wandb.agent(sweep_id, function=train)

wandb: Agent Starting Run: 00e6phxg with config:
	batch_size: 96
	epochs: 5
	layers: 96
wandb: Agent Started Run: 00e6phxg


Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 5s 80us/sample - loss: 0.8138 - accuracy: 0.6927 - val_loss: 0.6209 - val_accuracy: 0.7632
Epoch 2/5
60000/60000 [==============================] - 3s 44us/sample - loss: 0.5435 - accuracy: 0.7943 - val_loss: 0.5320 - val_accuracy: 0.8093
Epoch 3/5
60000/60000 [==============================] - 3s 45us/sample - loss: 0.4567 - accuracy: 0.8334 - val_loss: 0.4519 - val_accuracy: 0.8382
Epoch 4/5
60000/60000 [==============================] - 3s 44us/sample - loss: 0.3960 - accuracy: 0.8568 - val_loss: 0.4046 - val_accuracy: 0.8572
Epoch 5/5
60000/60000 [==============================] - 3s 45us/sample - loss: 0.3629 - accuracy: 0.8688 - val_loss: 0.3873 - val_accuracy: 0.8650
wandb: Agent Finished Run: 00e6phxg 

wandb: Agent Starting Run: zjpwkh0n with config:
	batch_size: 128
	epochs: 10
	layers: 256
wandb: Agent Started Run: zjpwkh0n


Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 5s 76us/sample - loss: 0.8146 - accuracy: 0.6908 - val_loss: 0.5999 - val_accuracy: 0.7707
Epoch 2/10
60000/60000 [==============================] - 2s 38us/sample - loss: 0.5381 - accuracy: 0.7993 - val_loss: 0.5113 - val_accuracy: 0.8164
Epoch 3/10
60000/60000 [==============================] - 2s 40us/sample - loss: 0.4581 - accuracy: 0.8341 - val_loss: 0.4586 - val_accuracy: 0.8381
Epoch 4/10
60000/60000 [==============================] - 2s 39us/sample - loss: 0.4081 - accuracy: 0.8517 - val_loss: 0.4111 - val_accuracy: 0.8522
Epoch 5/10
60000/60000 [==============================] - 2s 39us/sample - loss: 0.3758 - accuracy: 0.8636 - val_loss: 0.3888 - val_accuracy: 0.8624
Epoch 6/10
60000/60000 [==============================] - 2s 38us/sample - loss: 0.3439 - accuracy: 0.8738 - val_loss: 0.3997 - val_accuracy: 0.8516
Epoch 7/10
60000/60000 [==============================] 

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 5s 76us/sample - loss: 0.8590 - accuracy: 0.6746 - val_loss: 0.6227 - val_accuracy: 0.7618
Epoch 2/10
60000/60000 [==============================] - 3s 43us/sample - loss: 0.5519 - accuracy: 0.7926 - val_loss: 0.5335 - val_accuracy: 0.8059
Epoch 3/10
60000/60000 [==============================] - 3s 43us/sample - loss: 0.4722 - accuracy: 0.8271 - val_loss: 0.4756 - val_accuracy: 0.8293
Epoch 4/10
60000/60000 [==============================] - 3s 44us/sample - loss: 0.4180 - accuracy: 0.8485 - val_loss: 0.4310 - val_accuracy: 0.8486
Epoch 5/10
60000/60000 [==============================] - 3s 44us/sample - loss: 0.3837 - accuracy: 0.8604 - val_loss: 0.3979 - val_accuracy: 0.8608
Epoch 6/10
60000/60000 [==============================] - 3s 43us/sample - loss: 0.3533 - accuracy: 0.8732 - val_loss: 0.3750 - val_accuracy: 0.8696
Epoch 7/10
60000/60000 [==============================] 

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 4s 75us/sample - loss: 0.8712 - accuracy: 0.6771 - val_loss: 0.6293 - val_accuracy: 0.7545
Epoch 2/5
60000/60000 [==============================] - 2s 39us/sample - loss: 0.5581 - accuracy: 0.7896 - val_loss: 0.5346 - val_accuracy: 0.8099
Epoch 3/5
60000/60000 [==============================] - 2s 40us/sample - loss: 0.4781 - accuracy: 0.8259 - val_loss: 0.4876 - val_accuracy: 0.8297
Epoch 4/5
60000/60000 [==============================] - 2s 39us/sample - loss: 0.4251 - accuracy: 0.8462 - val_loss: 0.4234 - val_accuracy: 0.8507
Epoch 5/5
60000/60000 [==============================] - 2s 39us/sample - loss: 0.3886 - accuracy: 0.8602 - val_loss: 0.3986 - val_accuracy: 0.8593
wandb: Agent Finished Run: htj7hntx 

wandb: Agent Starting Run: 3svi4j41 with config:
	batch_size: 96
	epochs: 15
	layers: 96
wandb: Agent Started Run: 3svi4j41


...

Train on 60000 samples, validate on 10000 samples
Epoch 1/15
60000/60000 [==============================] - 6s 104us/sample - loss: 0.7871 - accuracy: 0.7018 - val_loss: 0.5899 - val_accuracy: 0.7777
Epoch 2/15
60000/60000 [==============================] - 4s 69us/sample - loss: 0.5082 - accuracy: 0.8122 - val_loss: 0.4903 - val_accuracy: 0.8258
Epoch 3/15
60000/60000 [==============================] - 4s 74us/sample - loss: 0.4192 - accuracy: 0.8487 - val_loss: 0.4367 - val_accuracy: 0.8483
Epoch 4/15
60000/60000 [==============================] - 4s 72us/sample - loss: 0.3708 - accuracy: 0.8664 - val_loss: 0.3902 - val_accuracy: 0.8569
Epoch 5/15
60000/60000 [==============================] - 4s 70us/sample - loss: 0.3409 - accuracy: 0.8761 - val_loss: 0.3507 - val_accuracy: 0.8750
Epoch 6/15
60000/60000 [==============================] - 4s 69us/sample - loss: 0.3173 - accuracy: 0.8842 - val_loss: 0.3571 - val_accuracy: 0.8719
Epoch 7/15
60000/60000 [==============================]

Train on 60000 samples, validate on 10000 samples
Epoch 1/15
60000/60000 [==============================] - 5s 77us/sample - loss: 0.8563 - accuracy: 0.6763 - val_loss: 0.6156 - val_accuracy: 0.7675
Epoch 2/15
60000/60000 [==============================] - 3s 43us/sample - loss: 0.5438 - accuracy: 0.7980 - val_loss: 0.5290 - val_accuracy: 0.8055
Epoch 3/15
60000/60000 [==============================] - 3s 44us/sample - loss: 0.4627 - accuracy: 0.8322 - val_loss: 0.4610 - val_accuracy: 0.8355
Epoch 4/15
60000/60000 [==============================] - 3s 43us/sample - loss: 0.4127 - accuracy: 0.8506 - val_loss: 0.4220 - val_accuracy: 0.8498
Epoch 5/15
60000/60000 [==============================] - 3s 44us/sample - loss: 0.3794 - accuracy: 0.8627 - val_loss: 0.3870 - val_accuracy: 0.8631
Epoch 6/15
60000/60000 [==============================] - 3s 42us/sample - loss: 0.3515 - accuracy: 0.8724 - val_loss: 0.3788 - val_accuracy: 0.8679
Epoch 7/15
60000/60000 [==============================] 

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 8s 137us/sample - loss: 0.6793 - accuracy: 0.7406 - val_loss: 0.5050 - val_accuracy: 0.8087
Epoch 2/10
60000/60000 [==============================] - 6s 104us/sample - loss: 0.4383 - accuracy: 0.8391 - val_loss: 0.4168 - val_accuracy: 0.8474
Epoch 3/10
60000/60000 [==============================] - 6s 100us/sample - loss: 0.3618 - accuracy: 0.8667 - val_loss: 0.3930 - val_accuracy: 0.8594
Epoch 4/10
60000/60000 [==============================] - 6s 101us/sample - loss: 0.3181 - accuracy: 0.8829 - val_loss: 0.3776 - val_accuracy: 0.8614
Epoch 5/10
60000/60000 [==============================] - 6s 102us/sample - loss: 0.2878 - accuracy: 0.8945 - val_loss: 0.3121 - val_accuracy: 0.8900
Epoch 6/10
60000/60000 [==============================] - 6s 101us/sample - loss: 0.2666 - accuracy: 0.9011 - val_loss: 0.3079 - val_accuracy: 0.8895
Epoch 7/10
60000/60000 [==========================

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 5s 77us/sample - loss: 0.8148 - accuracy: 0.6918 - val_loss: 0.6175 - val_accuracy: 0.7649
Epoch 2/10
60000/60000 [==============================] - 3s 43us/sample - loss: 0.5433 - accuracy: 0.7956 - val_loss: 0.5345 - val_accuracy: 0.8081
Epoch 3/10
60000/60000 [==============================] - 3s 43us/sample - loss: 0.4595 - accuracy: 0.8334 - val_loss: 0.4549 - val_accuracy: 0.8362
Epoch 4/10
60000/60000 [==============================] - 3s 43us/sample - loss: 0.4035 - accuracy: 0.8533 - val_loss: 0.4076 - val_accuracy: 0.8606
Epoch 5/10
60000/60000 [==============================] - 3s 43us/sample - loss: 0.3698 - accuracy: 0.8663 - val_loss: 0.4033 - val_accuracy: 0.8591
Epoch 6/10
60000/60000 [==============================] - 3s 44us/sample - loss: 0.3424 - accuracy: 0.8760 - val_loss: 0.3767 - val_accuracy: 0.8688
Epoch 7/10
60000/60000 [==============================] 

Train on 60000 samples, validate on 10000 samples
Epoch 1/15
60000/60000 [==============================] - 5s 77us/sample - loss: 0.8150 - accuracy: 0.6921 - val_loss: 0.6189 - val_accuracy: 0.7633
Epoch 2/15
60000/60000 [==============================] - 3s 43us/sample - loss: 0.5438 - accuracy: 0.7945 - val_loss: 0.5320 - val_accuracy: 0.8084
Epoch 3/15
60000/60000 [==============================] - 3s 43us/sample - loss: 0.4595 - accuracy: 0.8326 - val_loss: 0.4638 - val_accuracy: 0.8329
Epoch 4/15
60000/60000 [==============================] - 3s 43us/sample - loss: 0.4039 - accuracy: 0.8532 - val_loss: 0.4146 - val_accuracy: 0.8567
Epoch 5/15
60000/60000 [==============================] - 3s 44us/sample - loss: 0.3686 - accuracy: 0.8659 - val_loss: 0.3981 - val_accuracy: 0.8599
Epoch 6/15
60000/60000 [==============================] - 3s 44us/sample - loss: 0.3410 - accuracy: 0.8762 - val_loss: 0.3729 - val_accuracy: 0.8672
Epoch 7/15
60000/60000 [==============================] 

In [0]:
# Create a sweep for *Bayesian* search (run METHOD cell first and then train())
sweep_id = get_sweep_id('bayes')

Create sweep with ID: pu7llntm
Sweep URL: https://app.wandb.ai/sayakpaul/hyperparameter-sweeps-comparison/sweeps/pu7llntm


In [0]:
# Run the sweep
wandb.agent(sweep_id, function=train)

wandb: Agent Starting Run: s4a9kng3 with config:
	batch_size: 32
	epochs: 5
	layers: 64
wandb: Agent Started Run: s4a9kng3


Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 8s 134us/sample - loss: 0.6894 - accuracy: 0.7414 - val_loss: 0.4942 - val_accuracy: 0.8207
Epoch 2/5
60000/60000 [==============================] - 7s 109us/sample - loss: 0.4460 - accuracy: 0.8375 - val_loss: 0.4278 - val_accuracy: 0.8442
Epoch 3/5
60000/60000 [==============================] - 6s 104us/sample - loss: 0.3722 - accuracy: 0.8629 - val_loss: 0.3993 - val_accuracy: 0.8576
Epoch 4/5
60000/60000 [==============================] - 6s 100us/sample - loss: 0.3273 - accuracy: 0.8803 - val_loss: 0.3532 - val_accuracy: 0.8710
Epoch 5/5
60000/60000 [==============================] - 6s 101us/sample - loss: 0.2981 - accuracy: 0.8900 - val_loss: 0.3147 - val_accuracy: 0.8875
wandb: Agent Finished Run: s4a9kng3 

wandb: Agent Starting Run: 60afke8l with config:
	batch_size: 32
	epochs: 5
	layers: 256
wandb: Agent Started Run: 60afke8l


Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 8s 134us/sample - loss: 0.6587 - accuracy: 0.7496 - val_loss: 0.4868 - val_accuracy: 0.8201
Epoch 2/5
60000/60000 [==============================] - 6s 104us/sample - loss: 0.4224 - accuracy: 0.8456 - val_loss: 0.4293 - val_accuracy: 0.8445
Epoch 3/5
60000/60000 [==============================] - 6s 105us/sample - loss: 0.3484 - accuracy: 0.8714 - val_loss: 0.3735 - val_accuracy: 0.8649
Epoch 4/5
60000/60000 [==============================] - 6s 103us/sample - loss: 0.3053 - accuracy: 0.8878 - val_loss: 0.3713 - val_accuracy: 0.8644
Epoch 5/5
60000/60000 [==============================] - 6s 101us/sample - loss: 0.2767 - accuracy: 0.8981 - val_loss: 0.2958 - val_accuracy: 0.8892
wandb: Agent Finished Run: 60afke8l 

wandb: Agent Starting Run: 7tqasvfq with config:
	batch_size: 32
	epochs: 10
	layers: 256
wandb: Agent Started Run: 7tqasvfq


Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 8s 135us/sample - loss: 0.6598 - accuracy: 0.7479 - val_loss: 0.4903 - val_accuracy: 0.8212
Epoch 2/10
60000/60000 [==============================] - 6s 101us/sample - loss: 0.4229 - accuracy: 0.8459 - val_loss: 0.4187 - val_accuracy: 0.8488
Epoch 3/10
60000/60000 [==============================] - 6s 101us/sample - loss: 0.3486 - accuracy: 0.8716 - val_loss: 0.3830 - val_accuracy: 0.8606
Epoch 4/10
60000/60000 [==============================] - 6s 101us/sample - loss: 0.3041 - accuracy: 0.8886 - val_loss: 0.3567 - val_accuracy: 0.8714
Epoch 5/10
60000/60000 [==============================] - 6s 102us/sample - loss: 0.2751 - accuracy: 0.8989 - val_loss: 0.2880 - val_accuracy: 0.8976
Epoch 6/10
60000/60000 [==============================] - 6s 102us/sample - loss: 0.2553 - accuracy: 0.9061 - val_loss: 0.3056 - val_accuracy: 0.8935
Epoch 7/10
60000/60000 [==========================

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 6s 102us/sample - loss: 0.8098 - accuracy: 0.6967 - val_loss: 0.5915 - val_accuracy: 0.7764
Epoch 2/5
60000/60000 [==============================] - 4s 68us/sample - loss: 0.5290 - accuracy: 0.8048 - val_loss: 0.5004 - val_accuracy: 0.8228
Epoch 3/5
60000/60000 [==============================] - 4s 69us/sample - loss: 0.4410 - accuracy: 0.8411 - val_loss: 0.4552 - val_accuracy: 0.8386
Epoch 4/5
60000/60000 [==============================] - 4s 69us/sample - loss: 0.3898 - accuracy: 0.8592 - val_loss: 0.4007 - val_accuracy: 0.8611
Epoch 5/5
60000/60000 [==============================] - 4s 69us/sample - loss: 0.3581 - accuracy: 0.8707 - val_loss: 0.3758 - val_accuracy: 0.8683
wandb: Agent Finished Run: 3x16lnao 

wandb: Agent Starting Run: bjsvba5p with config:
	batch_size: 32
	epochs: 15
	layers: 256
wandb: Agent Started Run: bjsvba5p


...

Train on 60000 samples, validate on 10000 samples
Epoch 1/15
60000/60000 [==============================] - 8s 139us/sample - loss: 0.6597 - accuracy: 0.7478 - val_loss: 0.4910 - val_accuracy: 0.8198
Epoch 2/15
60000/60000 [==============================] - 6s 102us/sample - loss: 0.4207 - accuracy: 0.8459 - val_loss: 0.4185 - val_accuracy: 0.8479
Epoch 3/15
60000/60000 [==============================] - 6s 100us/sample - loss: 0.3473 - accuracy: 0.8717 - val_loss: 0.3872 - val_accuracy: 0.8578
Epoch 4/15
60000/60000 [==============================] - 6s 104us/sample - loss: 0.3050 - accuracy: 0.8871 - val_loss: 0.3654 - val_accuracy: 0.8696
Epoch 5/15
60000/60000 [==============================] - 7s 114us/sample - loss: 0.2776 - accuracy: 0.8975 - val_loss: 0.2948 - val_accuracy: 0.8951
Epoch 6/15
60000/60000 [==============================] - 6s 108us/sample - loss: 0.2570 - accuracy: 0.9046 - val_loss: 0.3193 - val_accuracy: 0.8863
Epoch 7/15
60000/60000 [==========================

Train on 60000 samples, validate on 10000 samples
Epoch 1/15
60000/60000 [==============================] - 8s 135us/sample - loss: 0.6584 - accuracy: 0.7491 - val_loss: 0.4912 - val_accuracy: 0.8194
Epoch 2/15
60000/60000 [==============================] - 6s 102us/sample - loss: 0.4212 - accuracy: 0.8457 - val_loss: 0.4145 - val_accuracy: 0.8489
Epoch 3/15
60000/60000 [==============================] - 6s 103us/sample - loss: 0.3481 - accuracy: 0.8719 - val_loss: 0.3871 - val_accuracy: 0.8587
Epoch 4/15
60000/60000 [==============================] - 6s 103us/sample - loss: 0.3056 - accuracy: 0.8874 - val_loss: 0.3775 - val_accuracy: 0.8633
Epoch 5/15
60000/60000 [==============================] - 6s 101us/sample - loss: 0.2775 - accuracy: 0.8979 - val_loss: 0.2986 - val_accuracy: 0.8915
Epoch 6/15
60000/60000 [==============================] - 6s 102us/sample - loss: 0.2570 - accuracy: 0.9056 - val_loss: 0.3124 - val_accuracy: 0.8896
Epoch 7/15
60000/60000 [==========================

Train on 60000 samples, validate on 10000 samples
Epoch 1/15
60000/60000 [==============================] - 8s 135us/sample - loss: 0.6576 - accuracy: 0.7497 - val_loss: 0.4886 - val_accuracy: 0.8196
Epoch 2/15
60000/60000 [==============================] - 6s 101us/sample - loss: 0.4207 - accuracy: 0.8457 - val_loss: 0.4192 - val_accuracy: 0.8479
Epoch 3/15
60000/60000 [==============================] - 6s 102us/sample - loss: 0.3474 - accuracy: 0.8715 - val_loss: 0.3786 - val_accuracy: 0.8636
Epoch 4/15
60000/60000 [==============================] - 6s 101us/sample - loss: 0.3043 - accuracy: 0.8877 - val_loss: 0.3594 - val_accuracy: 0.8685
Epoch 5/15
60000/60000 [==============================] - 6s 101us/sample - loss: 0.2758 - accuracy: 0.8980 - val_loss: 0.2987 - val_accuracy: 0.8911
Epoch 6/15
60000/60000 [==============================] - 6s 101us/sample - loss: 0.2555 - accuracy: 0.9053 - val_loss: 0.3096 - val_accuracy: 0.8923
Epoch 7/15
60000/60000 [==========================

Train on 60000 samples, validate on 10000 samples
Epoch 1/15
60000/60000 [==============================] - 8s 130us/sample - loss: 0.6602 - accuracy: 0.7479 - val_loss: 0.4887 - val_accuracy: 0.8184
Epoch 2/15
60000/60000 [==============================] - 6s 98us/sample - loss: 0.4204 - accuracy: 0.8462 - val_loss: 0.4166 - val_accuracy: 0.8491
Epoch 3/15
60000/60000 [==============================] - 6s 100us/sample - loss: 0.3448 - accuracy: 0.8730 - val_loss: 0.3774 - val_accuracy: 0.8627
Epoch 4/15
60000/60000 [==============================] - 6s 100us/sample - loss: 0.3027 - accuracy: 0.8882 - val_loss: 0.3869 - val_accuracy: 0.8576
Epoch 5/15
60000/60000 [==============================] - 6s 101us/sample - loss: 0.2754 - accuracy: 0.8980 - val_loss: 0.2954 - val_accuracy: 0.8923
Epoch 6/15
60000/60000 [==============================] - 6s 104us/sample - loss: 0.2547 - accuracy: 0.9064 - val_loss: 0.3108 - val_accuracy: 0.8912
Epoch 7/15
60000/60000 [===========================

Train on 60000 samples, validate on 10000 samples
Epoch 1/15
60000/60000 [==============================] - 8s 132us/sample - loss: 0.6761 - accuracy: 0.7426 - val_loss: 0.4946 - val_accuracy: 0.8150
Epoch 2/15
60000/60000 [==============================] - 6s 99us/sample - loss: 0.4352 - accuracy: 0.8401 - val_loss: 0.4187 - val_accuracy: 0.8472
Epoch 3/15
60000/60000 [==============================] - 6s 99us/sample - loss: 0.3595 - accuracy: 0.8677 - val_loss: 0.3949 - val_accuracy: 0.8611
Epoch 4/15
60000/60000 [==============================] - 6s 97us/sample - loss: 0.3155 - accuracy: 0.8833 - val_loss: 0.3822 - val_accuracy: 0.8617
Epoch 5/15
60000/60000 [==============================] - 6s 97us/sample - loss: 0.2863 - accuracy: 0.8946 - val_loss: 0.3064 - val_accuracy: 0.8895
Epoch 6/15
60000/60000 [==============================] - 6s 97us/sample - loss: 0.2649 - accuracy: 0.9019 - val_loss: 0.3141 - val_accuracy: 0.8894
Epoch 7/15
60000/60000 [==============================]

In [8]:
# Retrieve the train.py file
!wget https://raw.githubusercontent.com/sayakpaul/TF-2.0-Hacks/master/Bayesian_Search_W%26B/train.py
    
# Retrieve the sweep.yaml file
!wget https://raw.githubusercontent.com/sayakpaul/TF-2.0-Hacks/master/Bayesian_Search_W%26B/sweep.yaml

--2020-01-09 06:37:19--  https://raw.githubusercontent.com/sayakpaul/TF-2.0-Hacks/master/Bayesian_Search_W%26B/train.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2397 (2.3K) [text/plain]
Saving to: ‘train.py’

train.py            100%[===================>]   2.34K  --.-KB/s    in 0s      

2020-01-09 06:37:19 (32.9 MB/s) - ‘train.py’ saved [2397/2397]

--2020-01-09 06:37:21--  https://raw.githubusercontent.com/sayakpaul/TF-2.0-Hacks/master/Bayesian_Search_W%26B/sweep.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 348 [text/plain

In [11]:
# Initialize the sweep
!wandb sweep sweep.yaml

wandb: Creating sweep from: sweep.yaml
wandb: Created sweep with ID: i6u2qi5v
wandb: View sweep at: https://app.wandb.ai/sayakpaul/uncategorized/sweeps/i6u2qi5v
wandb: Run sweep agent with: wandb agent i6u2qi5v


In [12]:
# Run wandb agent to start the sweep
!wandb agent i6u2qi5v

wandb: Starting wandb agent 🕵️
2020-01-09 06:39:24,821 - wandb.wandb_agent - INFO - Running runs: []
2020-01-09 06:39:25,065 - wandb.wandb_agent - INFO - Agent received command: run
2020-01-09 06:39:25,065 - wandb.wandb_agent - INFO - Agent starting run with config:
	batch_size: 40
	epochs: 14
	layers: 176
	method: "grid"
wandb: Tracking run with wandb version 0.8.19
wandb: Run data is saved locally in wandb/run-20200109_063925-9zfclms5
wandb: Syncing run deep-sweep-1
wandb: ⭐️ View project at https://app.wandb.ai/sayakpaul/uncategorized
wandb: 🧹 View sweep at https://app.wandb.ai/sayakpaul/uncategorized/sweeps/i6u2qi5v
wandb: 🚀 View run at https://app.wandb.ai/sayakpaul/uncategorized/runs/9zfclms5
wandb: Run `wandb off` to turn off syncing.

2020-01-09 06:39:30,076 - wandb.wandb_agent - INFO - Running runs: ['9zfclms5']
2020-01-09 06:39:30.092566: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environ